##### Version for Multiple obj points (2 or 3) and multiple shifts (1, 2 or 3 now)
##### To run: chose proper os.chdir and data_path. chose proper input_dim, and note other hyper settings. Chose desired relative_pos_list. Rest doesn't need to change necessarily.

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split,Subset
from torch.nn.parameter import Parameter
from logger import Logger
import os
from utils_twopoints import *
from visualize import show2D_2objs, show2D_3objs
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
torch.backends.cudnn.deterministic=True

plt.rcParams.update({'font.size': 15,'lines.markersize':10})
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [ ]:
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 

#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_03 data')
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
#data_path='FS_LD9by9_All_CCD sweep xyz_avg_win1_normalizedbymax'

os.chdir('/home/zyhuang/WD/Net3D/2018_11_03 data/')
data_path='18_11_03sweepxyz_normalizedbymax'
        
scale_z=True #divide z coord by 33.3 to make it has same range as x,y
lr=1e-3
num_epoch=2000
logfreq=10
bs_test_percentage = 0.15
bs_train=100#remaining used as trainset
num_features=64
#input_dim=4*4*2
input_dim=4*4*2
nx,ny,nz=11,11,11

#relative_pos_list=[[(0,0,0),(1,2,3)]] # for two objs, single shift
relative_pos_list=[[(0,0,1),(0,3,0)],[(0,1,3),(2,1,1)]] # for two objs, two shift
#relative_pos_list=[[(0,0,1),(0,3,0)],[(0,1,3),(2,1,1)],[(1,3,2),(0,1,1)]] #for two objs, three shifts
#relative_pos_list=[[(0,0,1),(0,3,0),(1,0,2)],[(0,1,3),(2,1,1),(3,3,2)]] # for three objs, two shift
#relative_pos_list=[[(0,0,1),(0,3,0),(1,0,2)],[(0,1,3),(2,1,1),(3,3,2)],[(3,2,2),(0,2,1),(3,0,2)]] # for three objs, three shift
Nshift=len(relative_pos_list)
Nobj=len(relative_pos_list[0])
Nobj_eng={'2':'two','3':'three'}

if Nshift==1:
    relative_pos=relative_pos_list[0]
    ds_Full=globals()['FSdataset_'+Nobj_eng[str(Nobj)]+'points_v2'](data_path,relative_pos=relative_pos,nx=nx,ny=ny,nz=nz,scale_z=scale_z)
    bs_test = round(bs_test_percentage * len(ds_Full))
    save_name=str(Nobj)+'_objs_singlenetxyz_Set_loss_1shift_scale_z_'+str(scale_z)+'/'+"".join([s for s in str(relative_pos) if s.isdigit()])+'/depthnet4_Nobj_xyz('+str(num_features)+'features)'\
    +'_adam_lr_'+str(lr)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)
else:
    ds_Full=globals()['FSdataset_'+Nobj_eng[str(Nobj)]+'points_multipleshifts_v2'](data_path,relative_pos_list=relative_pos_list,nx=nx,ny=ny,nz=nz,scale_z=scale_z)
    bs_test = round(bs_test_percentage * len(ds_Full))
    save_name=str(Nobj)+'_objs_singlenetxyz_Set_loss_'+str(Nshift)+'shifts_scale_z_'+str(scale_z)+'/'+"".join([s for s in str(relative_pos_list) if s.isdigit()])\
    +'/depthnet4_Nobj_xyz('+str(num_features)+'features)'+'_adam_lr_'+str(lr)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)
print("Length of Full dataset is %s" %len(ds_Full))

ds_train,ds_test=random_split(ds_Full, [len(ds_Full)-bs_test,bs_test])   
train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=True)
test_loader=DataLoader(ds_test, batch_size=bs_test,shuffle=False)

logger = Logger('./logs_paper_data/'+ save_name) ##set the logger for tensorboard and save the log file to ./logs/Adam 
#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net=depthnet4_Nobj_xyz(input_dim,num_features,Nobj=Nobj)
net.type(dtype)

#criterion=torch.nn.L1Loss()
if Nobj == 2:
    criterion=Set_loss_2obj()   
elif Nobj == 3:
    criterion=Set_loss_3obj()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
loss_history=[]
test_loss_history=[]
for epoch in range(num_epoch):
    if np.mod(epoch,logfreq)==0: 
        print("Current epoch number%d" %epoch) 
    for idx,data in enumerate(train_loader,0):
        net.train()
        if Nobj == 2:
            FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
            xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
            loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
        elif Nobj == 3:
            FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
            xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
            loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)            
        loss_history.append(loss.data[0])
        if np.mod(epoch,logfreq)==0: 
            print('loss is %3f'%(loss.data[0]))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if np.mod(epoch,logfreq)==0:  
        net.eval()
        data=next(iter(test_loader))
        if Nobj ==2:
            FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
            xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
            test_loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
        elif Nobj ==3:
            FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
            xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
            test_loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)
            
        test_loss_history.append(test_loss.data[0])       
        print('Test Loss is %3f' %(test_loss.data[0]))#for multiple test sample
        
        step=epoch+1
        info = {
        'training_loss': loss.data[0],
        'val_loss':test_loss.data[0],
        }
        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)        

In [ ]:
plt.figure()
plt.semilogy(loss_history[::len(train_loader)],label='training_loss')#training_loss
plt.legend()
plt.title('Training loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Training loss')

plt.figure()
plt.semilogy(np.arange(0,epoch,logfreq),test_loss_history[:],label='test_loss')
plt.legend()
plt.title('Test loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Test loss')

In [ ]:
#this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
net.eval()

data=next(iter(test_loader))
if Nobj ==2:
    FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
    xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
elif Nobj ==3:
    FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
    xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)
text_file = open('logs_paper_data/'+save_name+"/Loss_final.txt", "w")
text_file.write('loss is %.4f(%.3f mm)\n'%(loss.data[0],np.sqrt(loss.data[0])))
text_file.write('Test Loss is %.4f(%.3f mm).' %(test_loss.data[0],np.sqrt(test_loss.data[0])))
text_file.close()
##

In [ ]:
test_bat=next(iter(test_loader))
FS_bat,trueXYZobjs_bat=test_bat['FS'],test_bat['xyz'][:,:]
#predXYZobjs_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()[:,[0,2,4,1,3,5]]
predXYZobjs_bat=net(Variable(FS_bat).type(dtype)).data.cpu()

#scale back the z coordinates
if Nobj == 2:
    predXYZobjs_bat[:,[2,5]] = predXYZobjs_bat[:,[2,5]]*33.3
    trueXYZobjs_bat[:,[2,5]] = trueXYZobjs_bat[:,[2,5]]*33.3
elif Nobj == 3:
    predXYZobjs_bat[:,[2,5,8]] = predXYZobjs_bat[:,[2,5,8]]*33.3
    trueXYZobjs_bat[:,[2,5,8]] = trueXYZobjs_bat[:,[2,5,8]]*33.3

print(predXYZobjs_bat)
print(trueXYZobjs_bat)

globals()['show2D_'+str(Nobj)+'objs'](trueXYZobjs_bat,predXYZobjs_bat,howmany=10,save_name='logs_paper_data/'+save_name+'/2Ds')   

In [ ]:
torch.save(net.state_dict(), os.path.join('logs_paper_data',save_name, 'model.pth'))
#Saving test data as .mat
from scipy import io
data={'trueXYZ':trueXYZobjs_bat.cpu().numpy(), 'PredictedXYZ':predXYZobjs_bat.cpu().numpy()}
io.savemat('logs_paper_data/'+save_name+"/testData.mat",data)

# Verify Test set and train set are not overlapping

In [ ]:
train_loader=DataLoader(ds_train, batch_size=710,shuffle=False)
test_loader=DataLoader(ds_test, batch_size=10,shuffle=False)
next(iter(train_loader))['xyz']

In [ ]:
#test if train/test overlap
bs_test=300 #specify test set size
bs_train=720-bs_test#remaining used as trainset
ds_Full=FSdataset_twopoints('18_11_03sweepxyz.mat',dx=1,dy=2,dz=3,nx=11,ny=11,nz=11)
ds_train,ds_test=random_split(ds_Full, [len(ds_Full)-bs_test,bs_test])   
train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=False)
test_loader=DataLoader(ds_test, batch_size=bs_test,shuffle=False)
next(iter(train_loader))['xyz']
((next(iter(test_loader))['xyz'][9]==next(iter(train_loader))['xyz']).sum(dim=1)==6).sum()